Unpickling data:
hr has reading every 5
steps has reading per minute, or every 12 elements. ! Not true due to missing readings


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn import preprocessing
import os
from collections import Counter
from datetime import date
import math
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
print("\n------------------------------------------------------\nStarting cycle\n------------------------------------------------------")
# Reference group average
ref_avg_accuracy_test = run([0, 1, 7, 8, 9, 10], 45)
print("Reference group done")

combinations = [
[0, 1, 7, 8], 
[0, 1, 7, 9], [0, 1, 7, 10],
[0, 1, 8, 9], [0, 1, 8, 10], [0, 1, 9, 10],
[0, 7, 8, 9], [0, 7, 8, 10], [0, 7, 9, 10],
[0, 8, 9, 10], [1, 7, 8, 9], [1, 7, 8, 10],
[1, 7, 9, 10], [1, 8, 9, 10], [7, 8, 9, 10]
]

# Heart failure group average
heart_failure_avg_accuracy_test = []

# (Very) fit group average
fit_avg_accuracy_test = []

# Obese group average
obese_avg_accuracy_test = []

# Paroxysmal atrial fibrillation group average
paroxysmal_atrial_fibrillation_avg_accuracy_test = []

runs_per_combination = 3

for run_index, combination in enumerate(combinations):
    # Heart failure group
    combination.extend([20, 24])
    if not np.any(heart_failure_avg_accuracy_test):
        heart_failure_avg_accuracy_test = run(combination, runs_per_combination)
    else:
        avg_accuracy_test = run(combination, runs_per_combination)
        heart_failure_avg_accuracy_test = [(x + y) / 2 for x, y in zip(heart_failure_avg_accuracy_test, avg_accuracy_test)]
    print("Run " + str(run_index) + "/" + str(runs_per_combination*len(combinations)) + " for 'heart failure' group done")

    combination.pop()  # Remove last element
    combination.pop()  # Remove last element

    # (Very) fit group
    combination.extend([2, 15])
    if not np.any(fit_avg_accuracy_test):
        fit_avg_accuracy_test = run(combination, runs_per_combination)
    else:
        avg_accuracy_test = run(combination, runs_per_combination)
        fit_avg_accuracy_test = [(x + y) / 2 for x, y in zip(fit_avg_accuracy_test, avg_accuracy_test)]
    print("Run " + str(run_index) + "/" + str(runs_per_combination*len(combinations)) + " for '(very) fit' group done")

    combination.pop()  # Remove last element
    combination.pop()  # Remove last element

    # Obese group
    combination.extend([3, 14])
    if not np.any(obese_avg_accuracy_test):
        obese_avg_accuracy_test = run(combination, runs_per_combination)
    else:
        avg_accuracy_test = run(combination, runs_per_combination)
        obese_avg_accuracy_test = [(x + y) / 2 for x, y in zip(obese_avg_accuracy_test, avg_accuracy_test)]
    print("Run " + str(run_index) + "/" + str(runs_per_combination*len(combinations)) + " for 'obese' group done")

    combination.pop()  # Remove last element
    combination.pop()  # Remove last element

    # Paroxysmal atrial fibrillation group
    combination.extend([29, 25])
    if not np.any(paroxysmal_atrial_fibrillation_avg_accuracy_test):
        paroxysmal_atrial_fibrillation_avg_accuracy_test = run(combination, runs_per_combination)
    else:
        avg_accuracy_test = run(combination, runs_per_combination)
        paroxysmal_atrial_fibrillation_avg_accuracy_test = [(x + y) / 2 for x, y in
                                                           zip(paroxysmal_atrial_fibrillation_avg_accuracy_test,
                                                               avg_accuracy_test)]
    print("Run " + str(run_index+1) + "/" + str(len(combinations)) + " for 'paroxysmal atrial fibrillation' group done")

    combination.pop()  # Remove last element
    combination.pop()  # Remove last element

print("\n------------------------------------------------------\nFull cycle done!\n------------------------------------------------------")

In [ ]:
plt.plot(ref_avg_accuracy_test, label='Reference group', color='blue')
plt.plot(heart_failure_avg_accuracy_test, label='Heart failure group', color='green')
plt.plot(fit_avg_accuracy_test, label='(Very) fit group', color='purple')
plt.plot(obese_avg_accuracy_test, label='Obese group', color='red')
plt.plot(paroxysmal_atrial_fibrillation_avg_accuracy_test, label='Paroxysmal atrial fibrillation group', color='orange')

plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.yticks(np.arange(0, 1, 0.1))
plt.legend(loc='lower right', fontsize=11)  # Move the legend to the bottom right

# Add horizontal raster lines
raster_values = np.arange(0, 1, 0.1)
for value in raster_values:
    plt.axhline(y=value, color='gray', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Define the variables and their corresponding file names
variables = [
    ('reference_hr_data', ['xxxxxx.pkl', 'xxxxxx.pkl',
                           'xxxxxx.pkl', 'xxxxxx.pkl',
                           'xxxxxx.pkl', 'xxxxxx.pkl']),
    ('fit_hr_data', ['xxxxxx.pkl', 'xxxxxx.pkl']),
    ('obese_hr_data', ['xxxxxx.pkl', 'xxxxxx.pkl']),
    ('heart_failure_hr_data', ['xxxxxx.pkl', 'xxxxxx.pkl']),
    ('paroxysmal_hr_data', ['xxxxxx.pkl', 'xxxxxx.pkl'])
]

# Create the figure and axis with smaller figsize
fig, ax = plt.subplots(figsize=(5.7, 4))
#fig.suptitle('Hourly Average Heart Rate')

# Define the colors for each group
colors = ['blue', 'purple', 'red', 'green', 'orange']

# Define the labels for each group
labels = ['Reference group', 'Fit group', 'Obese group', 'Heart failure group', 'Paroxysmal atrial fibrillation group']

# Iterate over the variables and plot the hourly averages
for (var_name, file_names), color, label in zip(variables, colors, labels):
    # Load and combine the variables from the files
    combined_variable = pd.DataFrame()
    for file_name in file_names:
        with open(file_name, 'rb') as f:
            variable = pkl.load(f)
        combined_variable = pd.concat([combined_variable, variable], ignore_index=True)

    # Convert the time column to a pandas DateTimeIndex
    combined_variable['time'] = pd.to_datetime(combined_variable['time'])

    # Calculate the hourly averages
    hourly_averages = combined_variable.groupby(combined_variable['time'].dt.hour)['hr'].mean()

    # Plot the hourly averages with labels
    ax.plot(hourly_averages.index, hourly_averages.values, label=label, color=color, linewidth=1)

# Set the x-axis label
ax.set_xlabel('Hour of the day')

# Set the y-axis label
ax.set_ylabel('Heart rate')

# Set the y-axis limits
ax.set_ylim(20, 90)

# Add horizontal raster lines
raster_values = np.arange(30, 90, 10)
for value in raster_values:
    plt.axhline(y=value, color='gray', linestyle='--', linewidth=0.5)

# Add a legend
ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
def run(keep, runs):
        
    # For reading in the pickled files

    subjects_dir = '/subjects/'
    subjects = os.listdir(subjects_dir)
    subjects_data = []
    meta_data = []
    s = []
    file_names = [
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl",
        "xxxxxx.pkl"
    ]

    # The following subjects have insufficient amount of data, so ignore them
    leave_out = [2, 8, 11, 12, 13, 16, 24, 27, 29, 30, 31, 32, 33, 34, 35, 39, 40, 42, 43, 46, 47, 48, 49]

    file_names = [file_names[i] for i in range(len(file_names)) if i not in leave_out]

    # This is to filter out to get specific cardiac diagnosis
    file_names = [file_names[i] for i in keep]

    file_map = {index: file_name for index, file_name in enumerate(file_names)}
    # Iterate over file_names and load subject data for valid indices
    for index, file_name in file_map.items():
        #if index not in leave_out:
        subject_file = os.path.join(subjects_dir, file_name)
        # Load subject data from file
        with open(subject_file, 'rb') as f:
            subject_data = pkl.load(f)
            s.append(file_name)
            subjects_data.append(subject_data)
        with open("meta.pkl", "rb") as f:
            meta_object = pkl.load(f)
            meta_data.append(meta_object)

    subjects = len(subjects_data)

    dx_cardiac_diagnosis_list = meta_object['dx_cardiac_diagnosis'].tolist()

    # Remove the last digit from each entry in data_platform_id
    data_platform_id_cleaned = [platform_id[:-1] if isinstance(platform_id, str) else platform_id for platform_id in meta_object['data_platform_id']]
    filtered_data_platform_id = [platform_id if platform_id in [name[:-4] for name in file_names] else None for platform_id in data_platform_id_cleaned]

    filtered_data_platform_id_length = len(filtered_data_platform_id)
    non_none_entries_count = len([entry for entry in filtered_data_platform_id if entry is not None])

    filtered_participant_id = [entry if str(entry) + ".pkl" in file_names else None for entry in meta_object['participant_id']]

    filtered_participant_id_length = len(filtered_participant_id)
    filtered_participant_id_count = sum(entry is not None for entry in filtered_participant_id)

    combined_ids_meta = [dp_id if dp_id is not None else p_id for dp_id, p_id in zip(filtered_data_platform_id, filtered_participant_id)]

    combined_ids_count = sum(1 for entry in combined_ids_meta if entry is not None)

    meta_indices = [combined_ids_meta.index(name[:-4]) if name[:-4] in combined_ids_meta else None for name in file_names]
    cardiac_status = []
    indices = []

    for index in meta_indices:
        if index is not None:
            cardiac_status.append(dx_cardiac_diagnosis_list[index])
            indices.append(index)
        else:
            cardiac_status.append(None)
            indices.append(None)

    frequency = Counter(cardiac_status)

    for entry, count in frequency.items():
        entry_indices = [i for i, status in enumerate(cardiac_status) if status == entry]

    minute = 12
    hour = minute * 60
    day = hour * 24 #17280

    def sliding_window (xs, window_size, stride):
        ps = []
        if len(xs) <= window_size:
            return []
        for i in range(len(xs) - window_size + 1):
            if i % stride == 0:
                ps.append(xs[i: i + window_size])
        return ps

    def normalize_vars(arr):
        valid_values = arr[~np.isnan(arr)]
        mean = np.mean(valid_values)
        sd = np.std(valid_values)
        return(mean, sd)

    def normalize(arr, mean, sd):
        templist = []
        for i in range(len(arr)):
            templist.append((arr[i] - mean) / sd)
        return templist

    test1 = np.array([1, 2, 3 ,4, 5, 6])
    vrs1 = normalize_vars(test1)
    test2 = np.array([np.NaN, np.NaN, np.NaN, 1, np.NaN, np.NaN])
    vrs2 = normalize_vars(test2)

    # Split data into days
    subjects = len(subjects_data) 
    subjects_data_by_day = []
    for i in range(subjects): # splits dataframe for each user into days
        df = subjects_data[i]
        day = df['time'].dt.floor("D")
        agg = df.groupby([day])
        smallList = []
        for i in agg:
            # Take only days of which there are enough readings! 
            if (len(i[1]) >= 14000): 
                steps = i[1].steps
                newsteps = steps[~np.isnan(steps)]
                smallList.append(i[1].head(14000)) 
        subjects_data_by_day.append(smallList)

    window_size = day 
    stride = window_size 
    balance = 50 

    identity = np.identity(subjects) # used for one hot encoding

    # Convert dataframe to np heart rate
    subjects_data_hr = []
    for i in range(subjects):
        smallList = []
        for j in range(len(subjects_data_by_day[i][:balance])):
            smallList.append(subjects_data_by_day[i][j].hr)
        subjects_data_hr.append(smallList)

    # Convert to train test
    X_train_users_temp = []
    X_test_users_temp = []
    Y_train_users = []
    Y_test_users = []

    for i in range(subjects):
        X_temp = np.array(subjects_data_hr[i])
        Y_temp = np.full(shape=len(X_temp), fill_value=i)
        X_train_temp, X_test_temp, Y_train_temp, Y_test_temp = train_test_split(X_temp, Y_temp, test_size=0.2, shuffle=False)

        X_train_users_temp.append(X_train_temp)
        X_test_users_temp.append(X_test_temp)


    # Normalize on variables from train set
    mean, sd = normalize_vars(np.concatenate(np.concatenate(X_train_users_temp)))
    X_train_users_hr = []
    X_test_users_hr = []
    for i in range(subjects):
        X_train_temp = np.array(normalize(X_train_users_temp[i], mean, sd))
        X_test_temp = np.array(normalize(X_test_users_temp[i], mean, sd))

        X_train_users_hr.append(X_train_temp)
        X_test_users_hr.append(X_test_temp)

    # Convert dataframe to np steps
    subjects_data_steps = []
    for i in range(subjects):
        smallList = []
        for j in range(len(subjects_data_by_day[i][:balance])):
            smallList.append(subjects_data_by_day[i][j].steps)
        subjects_data_steps.append(smallList)

    # Convert to train test
    X_train_users_temp = []
    X_test_users_temp = []
    Y_train_users = []
    Y_test_users = []
    for i in range(subjects):
        X_temp = np.array(subjects_data_steps[i])
        Y_temp = np.full(shape=len(X_temp), fill_value=i)
        X_train_temp, X_test_temp, Y_train_temp, Y_test_temp = train_test_split(X_temp, Y_temp, test_size=0.2, shuffle=False)

        X_train_users_temp.append(X_train_temp)
        X_test_users_temp.append(X_test_temp)

    # Normalize on variables from train set
    mean, sd = normalize_vars(np.concatenate(np.concatenate(X_train_users_temp)))

    X_train_users_steps_temp = []
    X_test_users_steps_temp = []
    for i in range(subjects):
        X_train_temp = np.array(normalize(X_train_users_temp[i], mean, sd))
        X_test_temp = np.array(normalize(X_test_users_temp[i], mean, sd))

        X_train_users_steps_temp.append(X_train_temp)
        X_test_users_steps_temp.append(X_test_temp)

    # Convert nan to unrealistic value
    X_train_users_steps = []
    X_test_users_steps = []
    for i in range(subjects):
        tempList = []
        for j in range(len(X_train_users_steps_temp[i])):
            window = X_train_users_steps_temp[i][j]
            window = np.where(np.isnan(window), mean, window)
            tempList.append(window)
        X_train_users_steps.append(tempList)
    for i in range(subjects):
        tempList = []
        for j in range(len(X_test_users_steps_temp[i])):
            window = X_test_users_steps_temp[i][j]
            window = np.where(np.isnan(window), mean, window)
            tempList.append(window)
        X_test_users_steps.append(tempList)

    # Apply sliding window
    X_train_hr = []  # List to store training windows
    X_train_steps = []  # List to store training windows
    X_test_hr = []  # List to store testing windows
    X_test_steps = []  # List to store testing windows
    Y_train_hr = []  # List to store training labels
    Y_train_steps = []  # List to store training labels
    Y_test_hr = []  # List to store testing labels
    Y_test_steps = []  # List to store testing labels
    train_len = int(balance * 0.8) #len(sliding_window(X_train_users_hr[10], window_size, stride)) # hardcoded :sad
    test_len = int(balance * 0.2) # len(sliding_window(X_test_users_hr[10], window_size, stride)) # also hardcoded :sad
    for j in range(train_len):
        for i in range(subjects):
            windows_train_hr = X_train_users_hr[i][j]
            windows_train_steps = X_train_users_steps[i][j]
            X_train_hr.append(np.array([windows_train_hr]))
            X_train_steps.append(np.array([windows_train_steps]))
            Y_train_hr.append(identity[i])
            Y_train_steps.append(identity[i])

    for j in range(test_len):
        for i in range(subjects):
            windows_test_hr = X_test_users_hr[i][j]
            windows_test_steps = X_test_users_steps[i][j]
            X_test_hr.append(np.array([windows_test_hr]))
            X_test_steps.append(np.array([windows_test_steps]))
            Y_test_hr.append(identity[i])
            Y_test_steps.append(identity[i])

    X_train_hr, X_val_hr, Y_train_hr, Y_val_hr = train_test_split(X_train_hr, Y_train_hr, test_size=0.2, random_state=42)
    X_train_steps, X_val_steps, Y_train_steps, Y_val_steps = train_test_split(X_train_steps, Y_train_steps, test_size=0.2, random_state=42)

    class CNN_LSTM(nn.Module):
        def __init__(self, in_channels, out_channels, kernel_size, stride, subjects):
            super(CNN_LSTM, self).__init__()

            self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride)
            self.relu1 = nn.ReLU()
            self.dropout1 = nn.Dropout(0.1)
            self.maxpool1 = nn.MaxPool1d(kernel_size=2)

            self.conv2 = nn.Conv1d(out_channels, 8, kernel_size=128, stride=32)
            self.relu2 = nn.ReLU()
            self.dropout2 = nn.Dropout(0.1)
            self.maxpool2 = nn.MaxPool1d(kernel_size=2)

            self.flatten = nn.Flatten()
            self.lstm = nn.LSTM(input_size=856, hidden_size=128, num_layers=1, batch_first=True)
            self.linear = nn.Linear(128, subjects)
            self.softmax = nn.Softmax(dim=1)

        def forward(self, x):
            x = self.conv1(x)
            x = self.relu1(x)
            x = self.dropout1(x)
            x = self.maxpool1(x)

            x = self.conv2(x)
            x = self.relu2(x)
            x = self.dropout2(x)
            x = self.maxpool2(x)

            x = self.flatten(x)
            x = x.view(x.size(0), 1, -1)  # Reshape the tensor to have a sequence dimension

            x, _ = self.lstm(x)
            x = x[:, -1, :]  # Select the last LSTM output of the sequence
            x = self.linear(x)
            x = self.softmax(x)

            return x

    # Set the number of runs
    num_runs = runs

    # Create empty lists to store the results of each run
    accuracy_train_all = []
    accuracy_val_all = []
    accuracy_test_all = []
    losses_all = []
    losses_val_all = []
    losses_test_all = []
    final_class_train_all = []
    final_class_test_all = []

    # Perform multiple runs
    for run in range(num_runs):

        # Hyperparameters
        input_size = 14000  
        lr = 0.001
        epochs = 50
        batch_size = 256

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Define the model architecture
        model = CNN_LSTM(in_channels=2, out_channels=4, kernel_size=5, stride=1, subjects=subjects).to(device)

        optimizer = optim.Adam(model.parameters(), lr=lr)
        loss_fn = nn.CrossEntropyLoss()

        accuracy_test = []
        losses = []
        accuracy_train = []
        accuracy_val = []


        accuracy = []
        losses = []
        losses_test = []
        losses_val = []
        accuracy_train = []

        # Training loop
        for epoch in range(epochs):
            correct_class_train = np.zeros(subjects)
            total_class_train = np.zeros(subjects)
            count_train = 0

            # Training
            model.train()

            class_correct_train = [0] * subjects
            class_total_train = [0] * subjects
            for i in range(0, len(X_train_hr), batch_size):
                remaining_samples = min(batch_size, len(X_train_hr) - i)
                X_batch_hr = torch.tensor(np.array(X_train_hr[i: i + remaining_samples]), dtype=torch.float32)
                X_batch_steps = torch.tensor(np.array(X_train_steps[i: i + remaining_samples]), dtype=torch.float32)
                Y_batch = torch.tensor(np.array(Y_train_hr[i: i + remaining_samples]))
                y_pred = model(torch.cat((X_batch_hr.to(device), X_batch_steps.to(device)), dim=1))

                loss = loss_fn(y_pred, Y_batch.to(device))
                optimizer.zero_grad()

                y_correct_train = np.argmax(Y_batch.detach().cpu().numpy(), axis=1)
                correct_class_train += np.sum(np.argmax(y_pred.detach().cpu().numpy(), axis=1) == y_correct_train)

                count_train += np.sum(np.argmax(y_pred.detach().cpu().numpy(), axis=1) == y_correct_train)

                loss.backward()
                optimizer.step()

                # Compute accuracy per class
                y_pred_labels = np.argmax(y_pred.detach().cpu().numpy(), axis=1)
                for j in range(len(y_correct_train)):
                    class_total_train[y_correct_train[j]] += 1
                    if y_correct_train[j] == y_pred_labels[j]:
                        class_correct_train[y_correct_train[j]] += 1

            losses.append(loss.item())
            accuracy_train.append(count_train / len(X_train_hr))

            # Compute accuracy per class
            final_class_train = [class_correct_train[i] / class_total_train[i] if class_total_train[i] > 0 else 0 for i in range(subjects)]

            # Validation
            model.eval()
            with torch.no_grad():
                correct_class_val = np.zeros(subjects)
                total_class_val = np.zeros(subjects)
                count_val = 0
                total_loss_val = 0

                for j in range(len(X_val_hr)):
                    X_batch_hr = np.reshape(X_val_hr[j], (1, 1, input_size))
                    X_batch_steps = np.reshape(X_val_steps[j], (1, 1, input_size))
                    tensor_hr = torch.tensor(X_batch_hr, dtype=torch.float32).to(device)
                    tensor_steps = torch.tensor(X_batch_steps, dtype=torch.float32).to(device)
                    y_pred = model(torch.cat((tensor_hr, tensor_steps), dim=1))
                    #y_pred = model(tensor_hr)
                    y_correct = np.argmax(Y_val_hr[j])
                    total_class_val[y_correct] += 1

                    if np.argmax(y_pred.detach().cpu().numpy()) == y_correct:
                        correct_class_val[y_correct] += 1
                        count_val += 1

                    # Compute loss for validation
                    Y_batch_val = torch.tensor(np.array([Y_val_hr[j]])).to(device)  # Wrap target in a list to match batch size
                    loss_val = loss_fn(y_pred, Y_batch_val)
                    total_loss_val += loss_val.item()

                losses_val.append(total_loss_val / len(X_val_hr))
                accuracy_val.append(count_val / len(X_val_hr))

            # Testing
            model.eval()
            with torch.no_grad():
                correct_class_test = np.zeros(subjects)
                total_class_test = np.zeros(subjects)
                count_test = 0
                total_loss_test = 0
                #print(len(X_test_hr)*2/6)
                for k in range(int(len(X_test_hr)*2/6)):
                    X_batch_hr = np.reshape(X_test_hr[k], (1, 1, input_size))
                    X_batch_steps = np.reshape(X_test_steps[k], (1, 1, input_size))
                    tensor_hr = torch.tensor(X_batch_hr, dtype=torch.float32).to(device)
                    tensor_steps = torch.tensor(X_batch_steps, dtype=torch.float32).to(device)
                    #y_pred = model(tensor_hr)
                    y_pred = model(torch.cat((tensor_hr, tensor_steps), dim=1))
                    #print(np.shape(tensor_hr))
                    y_correct = np.argmax(Y_test_hr[k])
                    total_class_test[y_correct] += 1

                    if np.argmax(y_pred.detach().cpu().numpy()) == y_correct:
                        correct_class_test[y_correct] += 1
                        count_test += 1

                    # Compute loss for testing
                    Y_batch_test = torch.tensor(np.array([Y_test_hr[k]])).to(device)  # Wrap target in a list to match batch size  
                    loss_test = loss_fn(y_pred, Y_batch_test)
                    total_loss_test += loss_test.item()

                losses_test.append(total_loss_test / int(len(X_test_hr)*2/6))
                accuracy_test.append(count_test / int(len(X_test_hr)*2/6))
                #print(model)

            final_class_test = correct_class_test / total_class_test

            #print(f"Run {run + 1}/{num_runs}")
            #print(f"Epoch {epoch + 1}/{epochs}:")
            #print(f"  Train Loss: {losses[-1]:.4f} - Train Accuracy: {accuracy_train[-1]*100:.2f}%")
            #print(f"  Validation Accuracy: {accuracy_val[-1]*100:.2f}%")
            #print(f"  Test Accuracy: {accuracy_test[-1]*100:.2f}%")
            #print()

            # Store the results of the current run
            accuracy_train_all.append(accuracy_train)
            accuracy_val_all.append(accuracy_val)
            accuracy_test_all.append(accuracy_test)
            losses_all.append(losses)
            losses_val_all.append(losses_val)
            losses_test_all.append(losses_test)
            final_class_train_all.append(final_class_train)
            final_class_test_all.append(final_class_test)

    # Convert the lists to NumPy arrays for easier averaging
    accuracy_train_all = np.array(accuracy_train_all)
    accuracy_val_all = np.array(accuracy_val_all)
    accuracy_test_all = np.array(accuracy_test_all)
    losses_all = np.array(losses_all)
    losses_val_all = np.array(losses_val_all)
    losses_test_all = np.array(losses_test_all)
    final_class_train_all = np.array(final_class_train_all)
    final_class_test_all = np.array(final_class_test_all)

    # Calculate the average results across runs
    avg_accuracy_train = np.mean(accuracy_train_all, axis=0)
    avg_accuracy_val = np.mean(accuracy_val_all, axis=0)
    avg_accuracy_test = np.mean(accuracy_test_all, axis=0)
    avg_losses = np.mean(losses_all, axis=0)
    avg_losses_val = np.mean(losses_val_all, axis=0)
    avg_losses_test = np.mean(losses_test_all, axis=0)

    # Calculate average testing accuracy across all runs
    avg_test_accuracy = np.mean(avg_accuracy_test) * 100
    
    return avg_accuracy_test